In [1]:
import os

import numpy as np
from sklearn.decomposition import PCA

from utils import load_hidden_representations_from_hdf5, read_templates_from_file

----

In [2]:
# params
log_dir = "/logfiles"
model = "bigscience-T0_3B" # bigscience-T0_B or bigscience-T0
module = "decoder"
task = "rte"
# task = "cb"
# task = "wic"

## Prompts

In [3]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_prompt.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_target_yes_no.csv")
display(df)

,name,template,category,includes_targets,targets,target_ids,shuffle
0,gpt_3_yes_no_with_targets,{premise} Question: {hypothesis} Yes or No?,instructive,True,"▁Yes, ▁No","0, 1",False
1,gpt_3_true_false_with_targets,{premise} Question: {hypothesis} True or False?,instructive,True,"▁True, ▁False","0, 1",False
2,gpt_3_cat_dog_with_targets,{premise} Question: {hypothesis} Cat or Dog?,instructive,True,"▁Cat, ▁Dog","0, 1",False
3,gpt_3_yes_no_without_targets,{premise} Question: {hypothesis}?,instructive,False,"▁Yes, ▁No","0, 1",False


## PCA

In [4]:
def unison_shuffled_copies(a, b):
    # from: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [5]:
if task == 'rte':
    use_pattern = [
        "gpt_3_yes_no_with_targets",
        "gpt_3_true_false_with_targets",
        "gpt_3_cat_dog_with_targets",
        "gpt_3_yes_no_without_targets",
    ]
elif task == 'cb':
    use_pattern = [
        "null_pattern",
        "null_pattern_reversed",
        "gpt_3_true_false_neither",
        "gpt_3_yes_no_maybe",
        "mnli_crowdsource",
        "always_sometimes_never",
        "based_on_previous_passage",
        "infer",
        "claim",
        "consider",
        "follow",
        "imply",
        "guaranteed",
        "guaranteed_possible",
        "justified",
        "must_be_true",
        "should_assume",
        "take_the_following",
    ]
elif task == 'wic':
    use_pattern = [
        "gpt_3",
        "gpt_3_yes_no",
        "affirmation",
        "grammar_homework",
        "polysemous",
        "question_context",
        "question_meaning",
        "question_meaning_yes_no",
        "same_sense",
        "similar_sense",
        "similar_sense_yes_no",
    ]

In [6]:
for t in range(2):
    # for layer in range(0, 10):
    for layer in range(0, 25):
    # for layer in range(24, 25):
        print(f"token: {t}\tlayer: {layer}")
        file_names, prompt_names = [], []

        for _, row in df.iterrows():
            if row['name'] in use_pattern:
                file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_t{t}_layer{layer}_avg.hdf5",)
                prompt_names.append(row['name'])


        # load hidden representations from hdf5 file
        representations = None
        classes = []
        n_sequences = 0

        for idx, file_name in enumerate(file_names):
            hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
            # print(hidden_representations.shape)
            n_sequences = hidden_representations.shape[0]

            if representations is None:
                representations = hidden_representations
            else:
                representations = np.concatenate((representations, hidden_representations), axis=0)

            classes += n_sequences * [idx] # assign representations to classes
        
        classes = np.asarray(classes)

        # shuffle representations and classes
        X, y = unison_shuffled_copies(representations, classes)
        print(X.shape, y.shape)

        # perform PCA on hidden representations
        print('PCA for prompts:', prompt_names)

        for n_components in range(1, 4):
            pca = PCA(n_components=n_components)
            pca.fit(X)

            # variance explained by each of the principal components
            print(f"model:{model}; module:{module}; token:{t}; layer:{layer}; n_components: {n_components}; variance explained: {pca.explained_variance_ratio_}")
            print(np.sum(pca.explained_variance_ratio_))
        print('\n')


token: 0	layer: 0


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5864.15it/s]
/opt/conda/lib/python3.6/site-packages/sklearn/decomposition/_pca.py:555: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ / total_var.sum()
/opt/conda/lib/python3.6/site-packages/sklearn/decomposition/_pca.py:555: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ / total_var.sum()
/opt/conda/lib/python3.6/site-packages/sklearn/decomposition/_pca.py:555: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ / total_var.sum()


(1108, 2048) (1108,)
PCA for prompts: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets', 'gpt_3_cat_dog_with_targets', 'gpt_3_yes_no_without_targets']
model:bigscience-T0_3B; module:decoder; token:0; layer:0; n_components: 1; variance explained: [nan]
nan
model:bigscience-T0_3B; module:decoder; token:0; layer:0; n_components: 2; variance explained: [nan nan]
nan
model:bigscience-T0_3B; module:decoder; token:0; layer:0; n_components: 3; variance explained: [nan nan nan]
nan


token: 0	layer: 1


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5681.59it/s]


(1108, 2048) (1108,)
PCA for prompts: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets', 'gpt_3_cat_dog_with_targets', 'gpt_3_yes_no_without_targets']
model:bigscience-T0_3B; module:decoder; token:0; layer:1; n_components: 1; variance explained: [0.9793358]
0.9793358
model:bigscience-T0_3B; module:decoder; token:0; layer:1; n_components: 2; variance explained: [0.9793358  0.00613705]
0.98547286
model:bigscience-T0_3B; module:decoder; token:0; layer:1; n_components: 3; variance explained: [0.9793358  0.00613705 0.00237561]
0.98784846


token: 0	layer: 2


...

(1108, 2048) (1108,)
PCA for prompts: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets', 'gpt_3_cat_dog_with_targets', 'gpt_3_yes_no_without_targets']
model:bigscience-T0_3B; module:decoder; token:1; layer:22; n_components: 1; variance explained: [0.57567465]
0.57567465
model:bigscience-T0_3B; module:decoder; token:1; layer:22; n_components: 2; variance explained: [0.5756748  0.14866985]
0.7243446
model:bigscience-T0_3B; module:decoder; token:1; layer:22; n_components: 3; variance explained: [0.57567495 0.14866976 0.06921825]
0.793563


token: 1	layer: 23


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 4432.37it/s]


(1108, 2048) (1108,)
PCA for prompts: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets', 'gpt_3_cat_dog_with_targets', 'gpt_3_yes_no_without_targets']
model:bigscience-T0_3B; module:decoder; token:1; layer:23; n_components: 1; variance explained: [0.6629282]
0.6629282
model:bigscience-T0_3B; module:decoder; token:1; layer:23; n_components: 2; variance explained: [0.66292816 0.13095386]
0.793882
model:bigscience-T0_3B; module:decoder; token:1; layer:23; n_components: 3; variance explained: [0.6629281  0.13095388 0.04393126]
0.83781326


token: 1	layer: 24


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5878.00it/s]


(1108, 2048) (1108,)
PCA for prompts: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets', 'gpt_3_cat_dog_with_targets', 'gpt_3_yes_no_without_targets']
model:bigscience-T0_3B; module:decoder; token:1; layer:24; n_components: 1; variance explained: [0.86665285]
0.86665285
model:bigscience-T0_3B; module:decoder; token:1; layer:24; n_components: 2; variance explained: [0.86665285 0.0853231 ]
0.95197594
model:bigscience-T0_3B; module:decoder; token:1; layer:24; n_components: 3; variance explained: [0.86665285 0.08532301 0.01280805]
0.96478385


